# k-NN Few-shot classification w/ GPT on Persuasive Essays

## "Single" approach

In [2]:
import os
import copy
import json
import time
import pickle
import random
from operator import itemgetter

import openai
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from openai.embeddings_utils import get_embedding
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from dotenv import load_dotenv

In [3]:
load_dotenv(verbose=True)

True

#### OpenAI Set-up

In [4]:
openai.api_type = "azure"
openai.api_base = "https://your.endpoint.url.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_KEY")  # This key should be in a local .env file

ENGINE_NAME='gpt-4'
EMBEDDING_ENGINE_NAME='ada-2'

In [8]:
df = pd.read_csv(os.path.join('data', 'pe_dataset.csv'))

In [9]:
df = df[df.columns[1:]]

In [10]:
df

,split,title,argument_bound_1,argument_bound_2,argument_id,ac,label,essay
0,TRAIN,Should students be taught to compete or to coo...,503,575,1,we should attach more importance to cooperatio...,MajorClaim,Should students be taught to compete or to coo...
1,TRAIN,Should students be taught to compete or to coo...,591,714,2,"through cooperation, children can learn about ...",Claim,Should students be taught to compete or to coo...
2,TRAIN,Should students be taught to compete or to coo...,716,851,3,What we acquired from team work is not only ho...,Premise,Should students be taught to compete or to coo...
3,TRAIN,Should students be taught to compete or to coo...,853,1086,4,"During the process of cooperation, children ca...",Premise,Should students be taught to compete or to coo...
4,TRAIN,Should students be taught to compete or to coo...,1088,1191,5,All of these skills help them to get on well w...,Premise,Should students be taught to compete or to coo...
...,...,...,...,...,...,...,...,...
6084,TRAIN,Children should studying hard or playing sport...,1275,1339,11,indirectly they will learn how to socialize ea...,Premise,Children should studying hard or playing sport...
6085,TRAIN,Children should studying hard or playing sport...,1341,1388,12,That will make children getting lots of friends,Premise,Children should studying hard or playing sport...
6086,TRAIN,Children should studying hard or playing sport...,1393,1436,13,they can contribute positively to community,Premise,Children should studying hard or playing sport...
6087,TRAIN,Children should studying hard or playing sport...,1448,1525,14,playing sport makes children getting healthy a...,Premise,Children should studying hard or playing sport...


#### Embed all essay titles

In [ ]:
title_embed_d = {}
for title in df.title.unique():
    print(title)
    while True:
        try:
            a = np.array(get_embedding(title, engine=EMBEDDING_ENGINE_NAME))
            title_embed_d[title] = a
            break
        except Exception as e:
            print(e)
            time.sleep(1)

In [ ]:
df['title_embedding'] = df.title.apply(lambda x: title_embed_d[x])

In [22]:
def get_k_neighbours(k, title, df):

    title_embed_d = {}
    for e in df.iterrows():
        if e[1].title not in title_embed_d:
            title_embed_d[e[1].title] = e[1].title_embedding

    train_titles = set(df[df.split == 'TRAIN'].title.unique())

    dist_l = []
    for t, v in title_embed_d.items():
        if t in train_titles:
            d = cosine(title_embed_d[title], v)
            dist_l.append((t, d))

    sorted_dist_l = sorted(dist_l, key=itemgetter(1))
    
    return sorted_dist_l[: k]

##### Test: Given a random title, fetch the 3 most similar titles.

In [23]:
get_k_neighbours(3, 'Detailed description of crimes on newspaper and TV can have bad consequences on society', df)

[('Large amount of violence in television programs', 0.13445186701270984),
 ('TV has adverse effects on friends and family', 0.1376546350015473),
 ('Television and Movies influence our lives both negatively and positively',
  0.14467761331309636)]

In [24]:
def prepare_similar_example_prompts(title, k=5):
    """
    Create a part of prompt made of k examples in the train set, whose topic is most similar to a given title.
    """

    neighbours_l = get_k_neighbours(k, title, df)

    prompt = ''
    cnt = 0
    for i, (title, dist) in enumerate(neighbours_l):
        prompt += f'# Example {i+1}\n'

        example_df = df[(df.split == 'TRAIN') & (df.title == title)]
        
        for k in example_df.iterrows():
            if k[1].argument_id == 1:
                prompt += f'Essay:\n{example_df.iloc[0].essay}\n\n'
                cnt = 0
                
            prompt += f'Argument {cnt + 1}={k[1].ac} - Class={k[1].label}\n'
            cnt += 1

        prompt += '\n\n'

    return prompt

##### Test: Print 3 similar examples passed to the prompt for essay 'International tourism is now more common than ever before'

In [25]:
print(prepare_similar_example_prompts('International tourism is now more common than ever before', k=3))

# Example 1
Essay:
International tourism is now more common than ever before

The last decade has seen an increasing number of tourists traveling to visit natural wonder sights, ancient heritages and different cultures around the world. While some people might think that this international tourism has negative effects on the destination countries, I would contend that it has contributed to the economic development as well as preserved the culture and environment of the tourist destinations. 
Firstly, international tourism promotes many aspects of the destination country’s economy in order to serve various demands of tourists. Take Cambodia for example, a large number of visitors coming to visit the Angkowat ancient temple need services like restaurants, hotels, souvenir shops and other stores. These demands trigger related business in the surrounding settings which in turn create many jobs for local people improve infrastructure and living standard. Therefore tourism has clearly improv

### Prepare system/user messages for all essays in test set

In [30]:
majorclaim_fulldesc = """The major claim represents the stance of the author about the essay topic. It is also called thesis statement and frequently indicated by opinion expressions like “From my point of view...”, “In my opinion...”, “I strongly believe that...”, etc. Usually, the major claim is present in the introduction or conclusion of an essay or in both. In the introduction it has the characteristics of a general assertion or an opinion with respect to the topic, whereas in the conclusion the major claim summarizes the argumentation according to the author’s stance."""

In [31]:
claim_fulldesc = "A claim in an body paragraph is the central component of an argument. It appears frequently as an initial assumption located at the beginning of a paragraph or as a conclusion near the end. In few cases, the claim might also be located somewhere between the statements of a paragraph. Most frequently, one paragraph includes a single unique claim and there are only few cases where several claims/arguments are included in a single body paragraph. In this case, a paragraph includes several arguments covering different topics or aspects related to the topic."

In [32]:
premise_fulldesc = "A premise is a reason given for supporting or attacking an argument component. So it can be considered as a justification or refutation for convincing the reader of the truth or falsity of a claim."

In [33]:
df[df.split == 'TRAIN'].label.value_counts(normalize=True)

label
Premise       0.626788
Claim         0.249222
MajorClaim    0.123989
Name: proportion, dtype: float64

In [34]:
proportion_desc = "62.7% of examples are of type Premise, 24.9% of type Claim, and 12.4% of type MajorClaim."

In [35]:
%%time

target_l = []
prepared_sys_task_msg_l = []

experiment_df = df[df.split == 'TEST']

# Pre-prepare all examples from each document
example_l = []
buffer_l = []
for e in experiment_df.iterrows():
    if e[1].argument_id == 1 and len(buffer_l) > 0:
        example_l.append(buffer_l)
        buffer_l = []

    buffer_l.append(e[1].ac)

example_l.append(buffer_l)

new_essay_l = []
cnt = 0
overall_cnt = -1
for i, e in enumerate(experiment_df.iterrows()):

    target = e[1].label

    if e[1].argument_id == 1:
        cnt = 0
        overall_cnt += 1
        new_essay_l.append(True)
    else:
        new_essay_l.append(False)

    # Prepare numbered list of ACs in this example
    other_acs_prompt = ''
    for j, s in enumerate(example_l[overall_cnt]):
        other_acs_prompt += f'Argument {j + 1}={s}\n'

    sys_msg = {"role":"system", "content": "### Task description: You are an expert assistant that takes 1) an essay, 2) the list of all arguments from this essay, 2) a current argument from this essay, and must classify this current argument into three classes: MajorClaim, Claim, and Premise. " + proportion_desc + " You must return a JSON with format {\"prediction\": <predicted class (str)>}\n\n### Class definitions:\nMajorClaim=" + majorclaim_fulldesc + "\n\nClaim=" + claim_fulldesc + "\n\nPremise=" + premise_fulldesc + "\n\n### Examples:\n" + prepare_similar_example_prompts(e[1].title, k=3)}  # Add proportion of classes
    task_msg = {"role":"user", "content": f"Essay:\n{e[1].essay}\n\n{other_acs_prompt}\n\nCurrent Argument {cnt + 1}={e[1].ac}"}

    prepared_sys_task_msg_l.append([sys_msg, task_msg])
    target_l.append(target)

    cnt += 1

CPU times: user 3min 27s, sys: 820 ms, total: 3min 28s
Wall time: 3min 28s


In [ ]:
random.seed(33)

idx_l = range(len(target_l))

subset_target_l = []
prediction_l = []
for i, idx in enumerate(idx_l):

    if i % 10 == 0:
        print(i, len(idx_l), accuracy_score(subset_target_l, prediction_l), f1_score(subset_target_l, prediction_l, average='macro'))

    task_sys_msg, task_task_msg = prepared_sys_task_msg_l[idx]

    cur_pred_l = []
    
    params_l = [(0.0, 0.1)]  # One call of temperature zero
    # params_l = [(0.0, 0.1), (0.4, 0.3), (0.7, 0.8)]  # Three calls of various temperatures/top_p
    # params_l = [(0.70, 0.80), (0.71, 0.81), (0.72, 0.82), (0.73, 0.83), (0.74, 0.84)]  # Five calls of close temperature/top_p
    for temp, top in params_l:

        while True:
    
            try:

                new_task_msg = task_task_msg.copy()

                response = openai.ChatCompletion.create(
                  engine=ENGINE_NAME,
                  messages = [task_sys_msg, new_task_msg],
                  temperature=temp,
                  max_tokens=20,
                  top_p=top,
                  frequency_penalty=0,
                  presence_penalty=0,
                  stop=None)
            
                prediction = response['choices'][0]['message']['content']                
                cur_pred_l.append(json.loads(prediction)['prediction'])
    
                break
    
            except Exception as e:
                print(e)
                time.sleep(5)

    mode_pred = max(set(cur_pred_l), key=cur_pred_l.count)
    prediction_l.append(mode_pred)
    subset_target_l.append(target_l[idx])
    

In [ ]:
print(classification_report(subset_target_l, prediction_l, digits=3))